### 1、ElasticSearch

* 概念：搜索引擎，还是数据库；
* 特点：
  * 1、分布式、支持多用户的全文搜索引擎。
  * **2、具有HTTP Web接口。**
  * **3、无模式JSON文档。**
  * 4、可以使用 **RESTful API 通过端口 9200 和 Elasticsearch 进行通信**

* **Stack Overflow 网站**，技术交流分享，解决bug的利器。
* **Elasticsearch 有2.x、5.x、6.x 三个大版本，我们在黑马头条中使用5.6版本。**



### 2、ES原理

* 搜索引擎原理 = **倒排索引 + 分析 + 相关性排序**

* **反向索引**：把列转成行，把字段转成记录

  * 1、正向索引：存储每个文档的单词，列表
  * select * from 诗词 where 内容=‘床头明月光，疑是地上霜，举头望明月，低头思故乡’

  | 序号 | 作者   | 名称     | 朝代   | 内容                                                         |
  | ---- | ------ | -------- | ------ | ------------------------------------------------------------ |
  | 1    | 李白   | 静夜思   | 唐     | 床头明月光，疑是地上霜，举头望明月，低头思故乡               |
  | 2    | 李绅   | 悯农     | 唐     | 锄禾日当午，汗滴禾下土，谁知盘中餐，粒粒皆辛苦               |
  | 3    | 岳飞   | 满江红   | 宋     | 怒发冲冠，凭栏处，潇潇雨歇，抬望眼，仰天长啸，壮怀激烈。。。 |
  | 4    | 朱自清 | 荷塘月色 | 近现代 | 曲曲折折的荷塘上面，弥望的是田田的叶子。。。                 |

  

  * 2、反向索引理解：存储古诗词，生活中搜索数据，都是模糊查找。

  * select * from 诗词表 where 记录=‘土’

    | 关键词 | 静夜思 | 悯农 | 满江红 | 荷塘月色 |
    | ------ | ------ | ---- | ------ | -------- |
    | 明月   | 1      | 0    | 0      | 1        |
    | 地上霜 | 1      | 0    | 0      | 0        |
    | 岳飞   | 0      | 0    | 1      | 0        |
    | 故乡   | 1      | 0    | 0      | 0        |
    | 土     | 0      | 1    | 1      | 1        |
    |        |        |      |        |          |

  

* 搜索引擎原理之**分析**：
  * 1、标准化：单复数、同义词、大小写；
  * 2、过滤：字符过滤器，分词器，Token 过滤器 

* 搜索引擎原理之**相关性排序**：
  * 1、检索词频率
  * 2、反向文档频率
  * 3、字段长度准则



### 3、ES相关概念理解

* 对比mysql：

~~~python
mysql:	数据库			表				列							 记录
ES：		 索引库		 类型			属性字段(域)			 文档  

~~~

* es集群：天生集群，类似于redis分布式集群(手动配置实现)；

~~~python
# 创建索引库，指定分片数目

PUT /Django
{
   "settings" : {
      "number_of_shards" : 3, # 索引库一旦建立，主分片不能修改
      "number_of_replicas" : 1 # 副本分片可以修改
   }
}
~~~



* curl指令：发送网络请求；

~~~python
# 向百度发送请求
curl www.baidu.com
# 请求方法、请求头、请求体
curl -X 请求方法 主机和端口 -H 请求头信息 -d 请求体信息
curl -X POST www.baidu.com -H 'Content-Type:application/json' -d '{key:value}'
# 查看集群运行状态,pretty表示美化输出
# yellow表示主分片运行正常，副本可能有异常
# red表示主分片异常
# green表示运行正常
curl -X GET 127.0.0.1:9200/_cluster/health?pretty
~~~



* 索引和类型：相当于mysql中的数据库和表；

  * 1、查看索引，show databases;

  ~~~python
  curl 127.0.0.1:9200/_cat/indices
  ~~~

  * 2、类型：ES5.x版本中的字符串类型为text，ES2.x版本中的字符串为string；
  * 3、工程开发中，数据库发生变化，代码也要因此修改操作方式，实际开发中如何避免，因为数据库发生变化，而需要修改代码。

  ~~~python
  ES中表中的字段数据类型不能修改，如果修改会报错，如何解决？
  关系型数据库可以通过迁移实现。
  # 老库articles,新库articles_v2
  ES中可以新建立索引库，将数据导入到新库。
  curl -X POST 127.0.0.1:9200/_reindex -H 'Content-Type:application/json' -d '
  {
    "source": {
      "index": "articles"
    },
    "dest": {
      "index": "articles_v2"
    }
  }
  '
  # 起别名
  from thread import Thread as t
  from process import Process as t
  # es中起别名
  articles_v1 as articles ----->代码中操作articles
  articles_v2 as articles ----->代码中操作articles
  
  curl -X DELETE 127.0.0.1:9200/articles
  curl -X PUT 127.0.0.1:9200/articles_v2/_alias/articles
  ~~~

  

* logstash工具导入数据：
  * 1、解压缩驱动：tar -zxvf mysqlxxxx
  * 2、vi mysql_logstash.conf，复制课件中的配置信息；
  * 3、执行指令：sudo /usr/share/logstash/bin/logstash -f ./logstash_mysql.conf
  * 不要执行时间太长，一直使用control + c暂停导入。

* 联想建议查询实现：
  * 1、查询es建议库，查询结果，如果有，返回列表
  * 2、前端展示建议结果
  * 3、如果es库中没有数据，需要查询suggest库，如果依然没有结果，返回None



### 4、单元测试

* 测试：检查预期结果和实际结果之间的差别。

* 单元测试的概念：对具体功能模块或接口的测试，属于测试大类中的一部分，只有单元测试可能会要求程序开发人员实现。
  * 本质：断言assert(python中的关键字)，断言的本质if 条件不满足，抛出异常。

* 单元测试的基本实现：项目中创建测试文件test
* 需要导入unittest模块

~~~python
import unittest
# 定义测试类，需要继承自己unittest.TestCase
class DatabaseTest(unittest.TestCase):
  
  # 函数名固定，相当于初始化函数，__init__函数
  # 会在测试代码执行前执行
  # 作用：在测试前做的准备工作，比如创建数据库、导入数据、构建测试客户端、初始化程序配置等。
  def setUp(self):
    self.client = self.app.test_client()
    app.config['SQLALCHEMY_DATABASE_URI'] = 'xxx' 
    pass
  
  # 函数名固定，相当于析构函数，__del__函数
  # 会在测试代码执行后执行
  # 作用：做测试后的扫尾工作，比如清除测试数据、移除数据库会话对象等。
  def tearDown(self):
    pass
  
  # 测试函数，函数名必须以test开头
  # 实现测试代码，比如发送请求、测试数据。
  def test_add_data(self):
    
    self.assert a == b
    pass

~~~



### 5、部署相关

* gunicorn：第三方应用服务器，能以多进程的方式启动项目；
* 安装：pip install gunicorn
* 启动：gunicorn -w 2 -b 绑定的主机和端口 启动文件:程序实例名

~~~python
# 阻塞执行
gunicorn -w 2 -b 0.0.0.0:8000 toutiao.main:app
# 后台运行，daemonnize
gunicorn -w 2 -b 0.0.0.0:8000 toutiao.main:app -D

~~~

* supervisor：进程管理工具，可以看管维护系统中的应用程序，如果发生异常，可以自动重启；
* **supervisor对python3支持不好，须使用python2**



